    Bastien KATUSZYNSKI - Maxime LEFEBVRE

## #1-Collecte des données

In [2]:
import sys
import random
import requests
import shutil
import os
import string
from pandas import json_normalize
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse

endpoint_url = "https://query.wikidata.org/sparql"
# Get cities
query = """SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
  ?grandeville wdt:P31 wd:Q1549591;
               wdt:P17 ?pays;
               wdt:P18 ?image.

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }
}
LIMIT 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["grandevilleLabel"]["value"],
            result["paysLabel"]["value"],
            result["image"]["value"],
        )
    )

dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)

def download_image(url):
    folder = "images/"
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        # Extract the filename from the URL and decode it
        filename = urllib.parse.unquote(os.path.basename(url))

        # Remove any query parameters from the filename
        filename, _ = os.path.splitext(filename)

        # Remove any special characters from the filename
        valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
        filename = ''.join(c for c in filename if c in valid_chars)

        # Extract the extension from the URL and add it to the filename
        extension = os.path.splitext(url)[1]
        filename += extension
        with open(folder + filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code


dataframe.image.apply(download_image)

0     200
1     200
2     200
3     200
4     200
     ... 
95    200
96    200
97    200
98    200
99    200
Name: image, Length: 100, dtype: int64

In [1]:
from PIL import Image
import os
import json
import sys
import random
import requests
import shutil
import string
from pandas import json_normalize
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse

# Chemin du dossier à parcourir
folder_path = "./images"

# Liste des propriétés de chaque image
images = []

# Parcours des fichiers du dossier
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    if os.path.isfile(filepath):
        # Vérifie si le fichier est une image
        if filepath.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
            # Ouverture de l'image
            with Image.open(filepath) as img:
                # Récupération des propriétés de l'image
                
                imgfile = Image.open(filepath)
            
                size = img.size
                format = img.format
                exif_data = imgfile._getexif()
                exif = "{\n"
                if exif_data:  # s'il existe des informations EXIF
                    num_tags = len(exif_data.items())
                    for i, (tag, value) in enumerate(exif_data.items()):
                        if tag in TAGS:
                            exif += "\""+str(TAGS[tag])+"\":"+str(value)
                            ## TODO Faire un if strTagsTag == Orientation et tous les autres hamps nécessaires alors récupéré la valeur et la formaté comme dans le append
                            if i != num_tags - 1:  # not the last tag
                                exif += ",\n"
                exif += "\n}"
                #print("exif",json.loads(exif))
                print("exif",exif)
                
                # Ajout des propriétés à la liste des images
                images.append({
                    "filename": filename,
                    "size": size,
                    "format": format,
                    #"exif": exif_data
                })
                

with open("images_informations.json", "w") as f:
    json.dump(images, f, indent=4, ensure_ascii=False)

NameError: name 'TAGS' is not defined